# **Planeación de rutas para la exploración en Marte**

Yuu Ricardo Akachi Tanaka | A01351969

Pablo Monzón Terrazas | A01562619

Donnet Emmanuel Hernández Franco | A01352049

In [ ]:
# ejecutar si no se tiene instalado simpleai
!pip install simpleai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=2850 # x inicial
        self.y=6400 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (3150,6800) ya escalado como en el mapa
        self.goal_state = (self.nr-round(6800/10.0174),round(3150/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#---------------------------------------------------------------------------------------------------------------
#   Programa
#---------------------------------------------------------------------------------------------------------------

#my_viewer = None
my_viewer = BaseViewer()       # Solo estadísticas
#my_viewer = ConsoleViewer()    # Texto en la consola
#my_viewer = WebViewer()        # Abrir en un browser en la liga http://localhost:8000




# Crea un PSA y lo resuelve con la búsqueda primero en anchura
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
Configuración inicial
   1176
   285
1 - Después de moverse hacia [1175, 285]
   1175
   285
2 - Después de moverse hacia [1174, 285]
   1174
   285
3 - Después de moverse hacia [1173, 285]
   1173
   285
4 - Después de moverse hacia [1172, 285]
   1172
   285
5 - Después de moverse hacia [1171, 285]
   1171
   285
6 - Después de moverse hacia [1170, 284]
   1170
   284
7 - Después de moverse hacia [1169, 284]
   1169
   284
8 - Después de moverse hacia [1168, 284]
   1168
   284
9 - Después de moverse hacia [1167, 284]
   1167
   284
10 - Después de moverse hacia [1166, 284]
   1166
   284
11 - Después de moverse hacia [1165, 283]
   1165
   283
12 - Después de moverse hacia [1164, 283]
   1164
   283
13 - Después de moverse hacia [1163, 283]
   1163
   283
14 - Después de moverse hacia [1162, 283]
   1162
   283
15 - Después de moverse hacia [1161, 283]
   1161
   283
16 - Después de moverse hacia [1160, 283]
   1160
   283
17 - Después de moverse haci

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda primero en anchura
result = breadth_first(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
Configuración inicial
   1176
   285
1 - Después de moverse hacia [1175, 285]
   1175
   285
2 - Después de moverse hacia [1174, 285]
   1174
   285
3 - Después de moverse hacia [1173, 285]
   1173
   285
4 - Después de moverse hacia [1172, 285]
   1172
   285
5 - Después de moverse hacia [1171, 285]
   1171
   285
6 - Después de moverse hacia [1170, 284]
   1170
   284
7 - Después de moverse hacia [1169, 284]
   1169
   284
8 - Después de moverse hacia [1168, 284]
   1168
   284
9 - Después de moverse hacia [1167, 284]
   1167
   284
10 - Después de moverse hacia [1166, 284]
   1166
   284
11 - Después de moverse hacia [1165, 283]
   1165
   283
12 - Después de moverse hacia [1164, 283]
   1164
   283
13 - Después de moverse hacia [1163, 283]
   1163
   283
14 - Después de moverse hacia [1162, 283]
   1162
   283
15 - Después de moverse hacia [1161, 283]
   1161
   283
16 - Después de moverse hacia [1160, 283]
   1160
   283
17 - Después de moverse haci

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda primero en anchura
result = depth_first(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')

KeyboardInterrupt: ignored

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda primero en anchura
result = greedy(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
Configuración inicial
   1176
   285
1 - Después de moverse hacia [1175, 286]
   1175
   286
2 - Después de moverse hacia [1174, 287]
   1174
   287
3 - Después de moverse hacia [1173, 287]
   1173
   287
4 - Después de moverse hacia [1172, 286]
   1172
   286
5 - Después de moverse hacia [1171, 286]
   1171
   286
6 - Después de moverse hacia [1171, 287]
   1171
   287
7 - Después de moverse hacia [1170, 287]
   1170
   287
8 - Después de moverse hacia [1169, 287]
   1169
   287
9 - Después de moverse hacia [1168, 286]
   1168
   286
10 - Después de moverse hacia [1167, 285]
   1167
   285
11 - Después de moverse hacia [1166, 284]
   1166
   284
12 - Después de moverse hacia [1165, 283]
   1165
   283
13 - Después de moverse hacia [1164, 283]
   1164
   283
14 - Después de moverse hacia [1163, 283]
   1163
   283
15 - Después de moverse hacia [1162, 283]
   1162
   283
16 - Después de moverse hacia [1161, 284]
   1161
   284
17 - Después de moverse haci

# Ruta corta

In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=4000 # x inicial
        self.y=17000 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (2400, 16000) ya escalado como en el mapa
        self.goal_state = (self.nr-round(17100/10.0174),round(4400/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda a*
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
Configuración inicial
   118
   399
1 - Después de moverse hacia [118, 400]
   118
   400
2 - Después de moverse hacia [118, 401]
   118
   401
3 - Después de moverse hacia [119, 402]
   119
   402
4 - Después de moverse hacia [120, 402]
   120
   402
5 - Después de moverse hacia [121, 402]
   121
   402
6 - Después de moverse hacia [122, 402]
   122
   402
7 - Después de moverse hacia [123, 403]
   123
   403
8 - Después de moverse hacia [123, 404]
   123
   404
9 - Después de moverse hacia [123, 405]
   123
   405
10 - Después de moverse hacia [122, 406]
   122
   406
11 - Después de moverse hacia [122, 407]
   122
   407
12 - Después de moverse hacia [122, 408]
   122
   408
13 - Después de moverse hacia [122, 409]
   122
   409
14 - Después de moverse hacia [121, 410]
   121
   410
15 - Después de moverse hacia [122, 410]
   122
   410
16 - Después de moverse hacia [123, 411]
   123
   411
17 - Después de moverse hacia [123, 412]
   123
   412
18 - D

In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=3800 # x inicial
        self.y=16500 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (2400, 16000) ya escalado como en el mapa
        self.goal_state = (self.nr-round(16900/10.0174),round(3900/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda a*
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
Configuración inicial
   168
   379
1 - Después de moverse hacia [167, 380]
   167
   380
2 - Después de moverse hacia [166, 381]
   166
   381
3 - Después de moverse hacia [166, 382]
   166
   382
4 - Después de moverse hacia [166, 383]
   166
   383
5 - Después de moverse hacia [165, 384]
   165
   384
6 - Después de moverse hacia [164, 384]
   164
   384
7 - Después de moverse hacia [163, 384]
   163
   384
8 - Después de moverse hacia [162, 384]
   162
   384
9 - Después de moverse hacia [161, 383]
   161
   383
10 - Después de moverse hacia [160, 384]
   160
   384
11 - Después de moverse hacia [159, 383]
   159
   383
12 - Después de moverse hacia [158, 383]
   158
   383
13 - Después de moverse hacia [157, 384]
   157
   384
14 - Después de moverse hacia [156, 384]
   156
   384
15 - Después de moverse hacia [155, 385]
   155
   385
16 - Después de moverse hacia [154, 385]
   154
   385
17 - Después de moverse hacia [153, 385]
   153
   385
18 - D

# Ruta mediana

In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=3800 # x inicial
        self.y=15000 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (2400, 16000) ya escalado como en el mapa
        self.goal_state = (self.nr-round(16500/10.0174),round(6000/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda a*
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
No se puede resolver
Stats:
{'max_fringe_size': 2035, 'visited_nodes': 640052, 'iterations': 640052}





In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=5000 # x inicial
        self.y=13000 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (2400, 16000) ya escalado como en el mapa
        self.goal_state = (self.nr-round(14500/10.0174),round(6000/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda a*
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
No se puede resolver
Stats:
{'max_fringe_size': 2035, 'visited_nodes': 72262, 'iterations': 72262}





# Ruta de mas de 10,000

In [ ]:
#---------------------------------------------------------------------------------------------------------------
#    PSA del robot explorador de Marte
#---------------------------------------------------------------------------------------------------------------

from simpleai.search import SearchProblem, depth_first, breadth_first, uniform_cost, greedy, astar
from simpleai.search.viewers import BaseViewer, ConsoleViewer, WebViewer
import numpy as np

#---------------------------------------------------------------------------------------------------------------
#   Definición del problema
#---------------------------------------------------------------------------------------------------------------

class Marte(SearchProblem):

    """
        Clase que es usada para definir el problema del robot explorador de Marte. Cada estado se representa
        por una coordenada.
    """

    def __init__(self):

        self.mapa = np.load('/content/mars_map.npy') # se crea el mapa de Marte
        self.nr, self.nc = self.mapa.shape # # se saca el tamaño del mapa
        self.x=5000 # x inicial
        self.y=16000 # y inicial
        self.r=self.nr-round(self.y/10.0174) # row escalado
        self.c=round(self.x/10.0174) # column escalado
        self.dif=0.25 # altura que puede bajar o subir

        initial_state = (self.r,self.c) # Estado (coordenada) en el que comienza el robot


        # Llama al constructor de su superclase SearchProblem.
        SearchProblem.__init__(self, initial_state)

        # Define el estado meta = que el robot termine en las coordenadas (2400, 16000) ya escalado como en el mapa
        self.goal_state = (self.nr-round(1400/10.0174),round(6000/10.0174))

    def actions(self, state):
        """
            Este método regresa una lista con las acciones posibles que pueden ser ejecutadas de
            acuerdo con el estado especificado.

            state: todas las coordenadas del mapa

            state[0] = row
            state[1] = column
        """

        acciones=[]
        #Movimiento hacia delante (respecto a la matriz)
        if self.mapa[state[0]][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]+1])<self.dif:
          a1=[state[0],state[1]+1]
          acciones.append(a1)
        #Movimiento hacia atras (respecto a la matriz)
        if self.mapa[state[0]][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]][state[1]-1])<self.dif:
          a2=[state[0],state[1]-1]
          acciones.append(a2)
        #Movimiento hacia la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]])<self.dif:
          a3=[state[0]+1,state[1]]
          acciones.append(a3)
        #Movimiento hacia la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]])<self.dif:
          a4=[state[0]-1,state[1]]
          acciones.append(a4)
        #Movimiento en diagonal hacia delante y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]+1])<self.dif:
          a5=[state[0]+1,state[1]+1]
          acciones.append(a5)
        #Movimiento en diagonal hacia delante y la izquierda
        if self.mapa[state[0]-1][state[1]+1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]+1])<self.dif:
          a6=[state[0]-1,state[1]+1]
          acciones.append(a6)
        #Movimiento en diagonal hacia atras y la derecha (respecto a la matriz)
        if self.mapa[state[0]+1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]+1][state[1]-1])<self.dif:
          a7=[state[0]+1,state[1]-1]
          acciones.append(a7)
        #Movimiento en diagonal hacia atras y la izquierda (respecto a la matriz)
        if self.mapa[state[0]-1][state[1]-1] !=-1 and abs(self.mapa[state[0]][state[1]]-self.mapa[state[0]-1][state[1]-1])<self.dif:
          a8=[state[0]-1,state[1]-1]
          acciones.append(a8)


        return acciones

    def result(self, state, action):
        """
            Este método regresa el nuevo estado obtenido despues de ejecutar la acción.

            state: el estado (coordenada) que se va a modificar
            action: movimiento que se hará en state (estado/coordenada)
        """
        return tuple([action[0],action[1]])

    def is_goal(self, state):
        """
            This method evaluates whether the specified state is the goal state.

            state: The state to be tested.
        """
        return state == self.goal_state

    def cost(self, state, action, state2):
        """
            Este método recibe dos estados y una acción, y regresa el costo de
            aplicar la acción del primer estado al segundo.

            state: Primer estado
            action: Acción usada para generar el segundo estado
            state2: Estado obtenido después de la acción
        """
        if abs(state2[0]-state[0]) == 1 and abs(state2[1]-state[1]) == 1:
          #moverse cada vez en diagonal tiene costo de 14.16
          return 14.16
        else:
          # moverse de otra forma tiene costo de 10.174
          return 10.0174


    def heuristic(self, state):
        """
            Este método regresa un estimado de la distancia desde el estado a la meta.

            state: Estado a ser evaluado

            La heurística es la distancia que le queda de llegar de donde está a la meta
        """
        distancia_diagonal=np.sqrt(((state[0]-self.goal_state[0])**2)+((state[1]-self.goal_state[1])**2))

        return distancia_diagonal


# Despliega los resultados
def display(result):
    if result is not None:
        for i, (action, state) in enumerate(result.path()):
            if action == None:
                print('Configuración inicial')
            elif i == len(result.path()) - 1:
                print(i,'- Después de moverse hacia', action)
                print('¡Meta lograda con costo =', result.cost,'!')
            else:
                print(i,'- Después de moverse hacia', action)

            for i in state:
              print('  ', i)
    else:
        print('No se puede resolver')

#---------------------------------------------------------------------------------------------------------------
#   Fin del archivo
#---------------------------------------------------------------------------------------------------------------

In [ ]:
# Crea un PSA y lo resuelve con la búsqueda a*
result = astar(Marte(), graph_search=True, viewer=my_viewer)
print()
print('>> Ruta hacia el punto destino')
display(result)
if my_viewer!=None:
    print('Stats:')
    print(my_viewer.stats)
print('\n\n')


>> Ruta hacia el punto destino
No se puede resolver
Stats:
{'max_fringe_size': 2035, 'visited_nodes': 72268, 'iterations': 72268}



